In [ ]:
import os
import glob
import shutil
import random

# Define your original dataset directory and the new structure
original_dataset_dir = '../.local/datasets/IoT_ObjectDetection'  # Path to your original dataset folder
new_dataset_dir = '../.local/datasets/IoT_ObjectDetection_Yolo'  # Path to the new structured dataset

In [ ]:
# Create new directories for train and validation
os.makedirs(os.path.join(new_dataset_dir, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, 'train', 'labels'), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(new_dataset_dir, 'val', 'labels'), exist_ok=True)

# Collect all class folders
class_folders = [d for d in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, d))]
class_folders.sort()  # Sort class folders to maintain consistent class index

# Create a mapping from class name to class index
class_mapping = {class_name: idx for idx, class_name in enumerate(class_folders)}

for class_name in class_folders:
    class_folder = os.path.join(original_dataset_dir, class_name)
    image_files = glob.glob(os.path.join(class_folder, '*.jpg'))

    # Randomly sample 10 images for validation
    val_samples = random.sample(image_files, min(10, len(image_files)))

    for img_path in image_files:
        # Get the base name without extension
        base_name = os.path.splitext(os.path.basename(img_path))[0]

        # Move the image to the train images folder if it's not in the validation sample
        if img_path not in val_samples:
            new_img_path = os.path.join(new_dataset_dir, 'train', 'images', os.path.basename(img_path))
            shutil.copy(img_path, new_img_path)

            # Move the corresponding label file
            label_file = os.path.join(class_folder, f"{base_name}.txt")
            if os.path.exists(label_file):
                new_label_path = os.path.join(new_dataset_dir, 'train', 'labels', f"{base_name}.txt")
                with open(label_file, 'r') as f:
                    with open(new_label_path, 'w') as f_out:
                        # Adjust the class index in the label file
                        for line in f.readlines():
                            parts = line.strip().split()
                            if len(parts) > 0:
                                # Replace the original class index with the correct index from the mapping
                                original_class_index = parts[0]
                                new_class_index = class_mapping[class_name]
                                # Write in YOLO format (new_class_index x_center y_center width height)
                                f_out.write(f"{new_class_index} {' '.join(parts[1:])}\n")

        else:
            # Move the selected validation image
            new_img_path = os.path.join(new_dataset_dir, 'val', 'images', os.path.basename(img_path))
            shutil.copy(img_path, new_img_path)

            # Move the corresponding validation label file
            label_file = os.path.join(class_folder, f"{base_name}.txt")
            if os.path.exists(label_file):
                new_label_path = os.path.join(new_dataset_dir, 'val', 'labels', f"{base_name}.txt")
                with open(label_file, 'r') as f:
                    with open(new_label_path, 'w') as f_out:
                        # Adjust the class index in the label file
                        for line in f.readlines():
                            parts = line.strip().split()
                            if len(parts) > 0:
                                # Replace the original class index with the correct index from the mapping
                                original_class_index = parts[0]
                                new_class_index = class_mapping[class_name]
                                # Write in YOLO format (new_class_index x_center y_center width height)
                                f_out.write(f"{new_class_index} {' '.join(parts[1:])}\n")

print("Dataset reorganized with random samples for validation and class indices adjusted!")

In [ ]:
import yaml

# Get class names from the folders in the original dataset directory
classes = [d for d in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, d))]
classes.sort()

data_dict = {
    'train': "IoT_ObjectDetection_Yolo/train",
    'val': "IoT_ObjectDetection_Yolo/val",
    'nc': len(classes),
    'names': classes,
}

dataset_yaml_path = 'data.yaml'
# Save the YAML file
with open(dataset_yaml_path, 'w') as f:
    yaml.dump(data_dict, f)

print("data.yaml file created!")

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')  # Change to the desired model variant

# Train the model
model.train(data=dataset_yaml_path, epochs=50, imgsz=640, batch=16)  # Adjust parameters as needed

print("Training completed!")

In [ ]:
# Save the trained model
model.save('yolov8s_trained.pt')

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s_trained.pt')
# Perform validation
results = model.val(data='data.yaml', save_json=True)  # Validate and save results in JSON format

In [32]:
import os
import glob
import yaml
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the YOLOv8 model
model = YOLO('yolov8s_trained.pt')

dataset_dir = "/home/tobias/Desktop/Uni/IMP/datasets/"

# predict on the validation set and visualize the results
with open('data.yaml') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
print(data['val'])
val_images = glob.glob(os.path.join(dataset_dir, data['val'], 'images', '*.jpg'))

for img_path in val_images:
    results = model(img_path)
    # Process results list
    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        obb = result.obb  # Oriented boxes object for OBB outputs
        result.show()  # display to screen
    

IoT_ObjectDetection_Yolo/val

image 1/1 /home/tobias/Desktop/Uni/IMP/datasets/IoT_ObjectDetection_Yolo/val/images/P1011441.jpg: 480x640 1 DeskLamp, 9.4ms
Speed: 2.2ms preprocess, 9.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/tobias/Desktop/Uni/IMP/datasets/IoT_ObjectDetection_Yolo/val/images/Tractorbot_69 (2).jpg: 640x384 2 Tractorbots, 10.4ms
Speed: 2.3ms preprocess, 10.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)
Opening in existing browser session.

image 1/1 /home/tobias/Desktop/Uni/IMP/datasets/IoT_ObjectDetection_Yolo/val/images/P1011378.jpg: 480x640 1 DeskLamp, 12.4ms
Speed: 3.2ms preprocess, 12.4ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Opening in existing browser session.

image 1/1 /home/tobias/Desktop/Uni/IMP/datasets/IoT_ObjectDetection_Yolo/val/images/P1011115.jpg: 480x640 1 OfficeDoor, 8.9ms
Opening in existing browser session.
Speed: 2.8ms preprocess, 8.9ms inference, 1.6ms postpr


KeyboardInterrupt

